# Konfiguration der Google Cloud und Herunterladen der JSON-Datei

In diesem Notebook wird die Google Cloud konfiguriert und eine JSON-Datei heruntergeladen.

In [13]:
# Importieren der benötigten Bibliotheken
from google.cloud import storage
from google.oauth2 import service_account

# Namen des Google Cloud Storage Buckets und der Quelldatei festlegen
bucket_name = 'prod_prototype'
source_blob_name = 'bronze/applicant_data_raw'
local_temp_path = '/tmp/applicant_data_raw.json'

# Dienstkonto-Datei laden
service_account_json = '/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Setup/prototyp-etl-pipline-d6cbb438aa70.json'
credentials = service_account.Credentials.from_service_account_file(service_account_json)

# Google Cloud Storage Client initialisieren
client = storage.Client(credentials=credentials, project='prototyp-etl-pipline')
bucket = client.bucket(bucket_name)
blob = bucket.blob(source_blob_name)

# Datei aus dem Bucket herunterladen
blob.download_to_filename(local_temp_path)
print(f"Datei erfolgreich heruntergeladen zu {local_temp_path}.")


Datei erfolgreich heruntergeladen zu /tmp/applicant_data_raw.json.


# Initialisiere Spark Session
In diesem Notebook wird die Spark Session vorbereitet.

In [14]:
# Importieren der benötigten Bibliotheken
from pyspark.sql import SparkSession

# Beispiel für das Hinzufügen des GCS Connectors zu einer lokalen Spark-Session
spark = SparkSession.builder \
    .appName("Datatransformation") \
    .config('spark.sql.debug.maxToStringFields', '1000') \
    .config("spark.jars.packages", "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.2") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Setup/prototyp-etl-pipline-d6cbb438aa70.json") \
    .getOrCreate()
    
# Überprüfen der SparkSession
spark

# Enttferung von Duplikaten
In diesem Notebook werdne alle doppelten Daten entfernt, leere Zellen bleiben leer

In [15]:
# JSON-Datei in ein Spark DataFrame laden
df = spark.read.json('/tmp/applicant_data_raw.json')

# Anzahl der ursprünglichen Datensätze
initial_count = df.count()

# Datenbereinigung: Entfernen von Duplikaten
df_cleaned = df.dropDuplicates()

# Anzahl der bereinigten Datensätze
cleaned_count = df_cleaned.count()

# Anzahl der gelöschten Duplikate
duplicates_removed = initial_count - cleaned_count
print(f"Anzahl der gelöschten Duplikate: {duplicates_removed}")

# Bereinigte Daten in eine neue Parquet-Datei speichern
cleaned_temp_path = '/tmp/applicant_data_cleaned.parquet'
df_cleaned.write.mode('overwrite').parquet(cleaned_temp_path)
print(f"Bereinigte Daten erfolgreich gespeichert zu {cleaned_temp_path}.")

Anzahl der gelöschten Duplikate: 0
Bereinigte Daten erfolgreich gespeichert zu /tmp/applicant_data_cleaned.parquet.


# Standartisierung Telefonnummern
In diesem Notebook werden Telefonnummern vereinheitlich

In [16]:
# Importieren der benötigten Bibliotheken
from pyspark.sql.functions import col, regexp_replace, length

# Lokaler Pfad zur bereinigten Parquet-Datei
cleaned_temp_path = '/tmp/applicant_data_cleaned.parquet'

# Parquet-Datei in ein Spark DataFrame laden
df_cleaned = spark.read.parquet(cleaned_temp_path)

# Überprüfen der vorhandenen Spalten
df_cleaned.printSchema()

# Sicherstellen, dass die Spalte 'phone' existiert
if 'Telefonnummer' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'phone' existiert nicht im DataFrame")

# Zählen der Telefonnummern vor der Transformation
initial_phone_count = df_cleaned.filter(length(col("Telefonnummer")) == 10).count()

# Umwandeln der Telefonnummern in das gewünschte Format: +1-245-345-7426
df_cleaned = df_cleaned.withColumn("phone", 
                                   regexp_replace(
                                       col("Telefonnummer"), 
                                       r"(\d{1})(\d{3})(\d{3})(\d{4})", 
                                       r"+1-$2-$3-$4"
                                   ))

# Zählen der Telefonnummern nach der Transformation
transformed_phone_count = df_cleaned.filter(col("Telefonnummer").like("+1-%-%-%")).count()

# Anzahl der durchgeführten Transformationen
transformations_done = transformed_phone_count
print(f"Anzahl der durchgeführten Transformationen: {transformations_done}")

# Bereinigte Daten in eine neue Parquet-Datei speichern
transformed_temp_path = '/tmp/applicant_data_transformed.parquet'
df_cleaned.write.mode('overwrite').parquet(transformed_temp_path)
print(f"Transformierte Daten erfolgreich gespeichert zu {transformed_temp_path}.")

root
 |-- Ablehnungsgrund: string (nullable = true)
 |-- Adresse: string (nullable = true)
 |-- Bewerbungsdatum: string (nullable = true)
 |-- Bewerbungsquelle: string (nullable = true)
 |-- Bewertung Assessment: string (nullable = true)
 |-- Bewertung Erstes Gespräch: string (nullable = true)
 |-- Bewertung Prescreening: string (nullable = true)
 |-- E-Mail: string (nullable = true)
 |-- Effektiver Gehalt: double (nullable = true)
 |-- Einstellungsdatum: string (nullable = true)
 |-- Geburtsdatum: string (nullable = true)
 |-- Gehaltsvorstellungen: string (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- Headhunter Firma: string (nullable = true)
 |-- Headhunter Name: string (nullable = true)
 |-- Interviewer: string (nullable = true)
 |-- Job Titel: string (nullable = true)
 |-- Kandidaten-ID: string (nullable = true)
 |-- Nachname: string (nullable = true)
 |-- Standort: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- StellenID: string (nullable = 

# Auffüllen leerer Adresszellen

In diesem Notebook werden leere Zellen in der Spalte Adresse mit dem Platzhalter "Unknown" gefüllt.


In [17]:
# Importieren der benötigten Bibliotheken
from pyspark.sql.functions import col, when

# Lokaler Pfad zur bereinigten Parquet-Datei
transformed_temp_path = '/tmp/applicant_data_transformed.parquet'

# Parquet-Datei in ein Spark DataFrame laden
df_cleaned = spark.read.parquet(transformed_temp_path)

# Überprüfen der vorhandenen Spalten
df_cleaned.printSchema()

# Sicherstellen, dass die Spalte 'Adresse' existiert
if 'Adresse' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Adresse' existiert nicht im DataFrame")

# Zählen der leeren Adresszellen vor der Transformation
initial_empty_address_count = df_cleaned.filter(col("Adresse").isNull()).count()

# Auffüllen leerer Zellen in der Spalte 'Adresse' mit 'Unknown'
df_cleaned = df_cleaned.withColumn('Adresse', when(col('Adresse').isNull(), 'Unknown').otherwise(col('Adresse')))

# Zählen der leeren Adresszellen nach der Transformation
final_empty_address_count = df_cleaned.filter(col("Adresse") == 'Unknown').count()

# Anzahl der durchgeführten Auffüllungen
fillings_done = final_empty_address_count
print(f"Anzahl der aufgefüllten Adresszellen: {fillings_done}")

# Bereinigte Daten in eine neue Parquet-Datei speichern
final_temp_path = '/tmp/applicant_data_final.parquet'
df_cleaned.write.mode('overwrite').parquet(final_temp_path)
print(f"Bereinigte Daten erfolgreich gespeichert zu {final_temp_path}.")


root
 |-- Ablehnungsgrund: string (nullable = true)
 |-- Adresse: string (nullable = true)
 |-- Bewerbungsdatum: string (nullable = true)
 |-- Bewerbungsquelle: string (nullable = true)
 |-- Bewertung Assessment: string (nullable = true)
 |-- Bewertung Erstes Gespräch: string (nullable = true)
 |-- Bewertung Prescreening: string (nullable = true)
 |-- E-Mail: string (nullable = true)
 |-- Effektiver Gehalt: double (nullable = true)
 |-- Einstellungsdatum: string (nullable = true)
 |-- Geburtsdatum: string (nullable = true)
 |-- Gehaltsvorstellungen: string (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- Headhunter Firma: string (nullable = true)
 |-- Headhunter Name: string (nullable = true)
 |-- Interviewer: string (nullable = true)
 |-- Job Titel: string (nullable = true)
 |-- Kandidaten-ID: string (nullable = true)
 |-- Nachname: string (nullable = true)
 |-- Standort: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- StellenID: string (nullable = 

# Auffüllen leerer Datumszellen

In diesem Notebook werden leere Zellen in den Spalten Geburtsdatum, Bewerbungsdatum, Veröffentlichungsdatum und Einstellungsdatum mit dem Platzhalter "1900-01-01" gefüllt.

In [18]:
# Importieren der benötigten Bibliotheken
from pyspark.sql.functions import col, when

# Lokaler Pfad zur bereinigten Parquet-Datei
final_temp_path = '/tmp/applicant_data_final.parquet'

# Parquet-Datei in ein Spark DataFrame laden
df_cleaned = spark.read.parquet(final_temp_path)

# Überprüfen der vorhandenen Spalten
df_cleaned.printSchema()

# Sicherstellen, dass die Spalten 'Geburtsdatum', 'Bewerbungsdatum', 'Veröffentlichungsdatum' und 'Einstellungsdatum' existieren
if 'Geburtsdatum' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Geburtsdatum' existiert nicht im DataFrame")
if 'Bewerbungsdatum' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Bewerbungsdatum' existiert nicht im DataFrame")
if 'Veröffentlichungsdatum' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Veröffentlichungsdatum' existiert nicht im DataFrame")
if 'Einstellungsdatum' not in df_cleaned.columns:
    raise ValueError("Die Spalte 'Einstellungsdatum' existiert nicht im DataFrame")

# Zählen der leeren Datumszellen vor der Transformation
initial_empty_birthdate_count = df_cleaned.filter(col("Geburtsdatum").isNull()).count()
initial_empty_applicationdate_count = df_cleaned.filter(col("Bewerbungsdatum").isNull()).count()
initial_empty_publishdate_count = df_cleaned.filter(col("Veröffentlichungsdatum").isNull()).count()
initial_empty_hiredate_count = df_cleaned.filter(col("Einstellungsdatum").isNull()).count()

# Auffüllen leerer Zellen in den Spalten mit '1900-01-01'
df_cleaned = df_cleaned.withColumn('Geburtsdatum', when(col('Geburtsdatum').isNull(), '1900-01-01').otherwise(col('Geburtsdatum')))
df_cleaned = df_cleaned.withColumn('Bewerbungsdatum', when(col('Bewerbungsdatum').isNull(), '1900-01-01').otherwise(col('Bewerbungsdatum')))
df_cleaned = df_cleaned.withColumn('Veröffentlichungsdatum', when(col('Veröffentlichungsdatum').isNull(), '1900-01-01').otherwise(col('Veröffentlichungsdatum')))
df_cleaned = df_cleaned.withColumn('Einstellungsdatum', when(col('Einstellungsdatum').isNull(), '1900-01-01').otherwise(col('Einstellungsdatum')))

# Zählen der leeren Datumszellen nach der Transformation
final_empty_birthdate_count = df_cleaned.filter(col("Geburtsdatum") == '1900-01-01').count()
final_empty_applicationdate_count = df_cleaned.filter(col("Bewerbungsdatum") == '1900-01-01').count()
final_empty_publishdate_count = df_cleaned.filter(col("Veröffentlichungsdatum") == '1900-01-01').count()
final_empty_hiredate_count = df_cleaned.filter(col("Einstellungsdatum") == '1900-01-01').count()

# Anzahl der durchgeführten Auffüllungen
birthdate_fillings_done = final_empty_birthdate_count
applicationdate_fillings_done = final_empty_applicationdate_count
publishdate_fillings_done = final_empty_publishdate_count
hiredate_fillings_done = final_empty_hiredate_count

print(f"Anzahl der aufgefüllten Geburtsdatum Zellen: {birthdate_fillings_done}")
print(f"Anzahl der aufgefüllten Bewerbungsdatum Zellen: {applicationdate_fillings_done}")
print(f"Anzahl der aufgefüllten Veröffentlichungsdatum Zellen: {publishdate_fillings_done}")
print(f"Anzahl der aufgefüllten Einstellungsdatum Zellen: {hiredate_fillings_done}")

# Bereinigte Daten in eine neue Parquet-Datei speichern
final_temp_path_updated = '/tmp/applicant_data_final_updated.parquet'
df_cleaned.write.mode('overwrite').parquet(final_temp_path_updated)
print(f"Bereinigte Daten erfolgreich gespeichert zu {final_temp_path_updated}.")


root
 |-- Ablehnungsgrund: string (nullable = true)
 |-- Adresse: string (nullable = true)
 |-- Bewerbungsdatum: string (nullable = true)
 |-- Bewerbungsquelle: string (nullable = true)
 |-- Bewertung Assessment: string (nullable = true)
 |-- Bewertung Erstes Gespräch: string (nullable = true)
 |-- Bewertung Prescreening: string (nullable = true)
 |-- E-Mail: string (nullable = true)
 |-- Effektiver Gehalt: double (nullable = true)
 |-- Einstellungsdatum: string (nullable = true)
 |-- Geburtsdatum: string (nullable = true)
 |-- Gehaltsvorstellungen: string (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- Headhunter Firma: string (nullable = true)
 |-- Headhunter Name: string (nullable = true)
 |-- Interviewer: string (nullable = true)
 |-- Job Titel: string (nullable = true)
 |-- Kandidaten-ID: string (nullable = true)
 |-- Nachname: string (nullable = true)
 |-- Standort: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- StellenID: string (nullable = 

# Hochladen der bereinigten Daten in den Google Cloud Storage

In diesem Notebook werden die bereinigten Daten wieder in den Google Cloud Storage hochgeladen.


In [19]:
# Importieren der benötigten Bibliotheken
import os

# Funktion zum Hochladen eines Verzeichnisses zu Google Cloud Storage
def upload_directory_to_gcs(bucket_name, source_directory, destination_blob_prefix):
    client = storage.Client(credentials=credentials, project='prototyp-etl-pipline')
    bucket = client.bucket(bucket_name)
    
    for root, dirs, files in os.walk(source_directory):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, source_directory)
            blob_path = os.path.join(destination_blob_prefix, relative_path)
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_path)
            print(f"File {local_path} uploaded to {blob_path}.")

# Namen des Google Cloud Storage Buckets und der bereinigten Datei festlegen
bucket_name = 'prod_prototype'
cleaned_blob_prefix = 'sensitive/datenbereinigung'
cleaned_temp_path = '/tmp/applicant_data_cleaned.parquet'

# Dienstkonto-Datei laden
service_account_json = '/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Setup/prototyp-etl-pipline-d6cbb438aa70.json'
credentials = service_account.Credentials.from_service_account_file(service_account_json)

# Hochladen des Parquet-Verzeichnisses
upload_directory_to_gcs(bucket_name, cleaned_temp_path, cleaned_blob_prefix)
print(f"Bereinigte Dateien erfolgreich hochgeladen zu {cleaned_blob_prefix} in bucket {bucket_name}.")

# Lokaler Pfad zur CSV-Datei
csv_output_path = '/Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Output/applicant_data_final.csv'

# DataFrame in eine CSV-Datei speichern
df_cleaned.write.mode('overwrite').option("header", "true").csv(csv_output_path)
print(f"Bereinigte Daten erfolgreich in CSV gespeichert zu {csv_output_path}.")


File /tmp/applicant_data_cleaned.parquet/._SUCCESS.crc uploaded to sensitive/datenbereinigung/._SUCCESS.crc.
File /tmp/applicant_data_cleaned.parquet/part-00000-4fe844df-d8f0-4280-bbb4-bd0da92f1480-c000.snappy.parquet uploaded to sensitive/datenbereinigung/part-00000-4fe844df-d8f0-4280-bbb4-bd0da92f1480-c000.snappy.parquet.
File /tmp/applicant_data_cleaned.parquet/.part-00000-4fe844df-d8f0-4280-bbb4-bd0da92f1480-c000.snappy.parquet.crc uploaded to sensitive/datenbereinigung/.part-00000-4fe844df-d8f0-4280-bbb4-bd0da92f1480-c000.snappy.parquet.crc.
File /tmp/applicant_data_cleaned.parquet/_SUCCESS uploaded to sensitive/datenbereinigung/_SUCCESS.
Bereinigte Dateien erfolgreich hochgeladen zu sensitive/datenbereinigung in bucket prod_prototype.
Bereinigte Daten erfolgreich in CSV gespeichert zu /Users/Kevin/Documents/GitHub/Transferarbeit/Prototyp_Transferarbeit_Lokal/Output/applicant_data_final.csv.


24/06/17 01:14:50 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2653062 ms exceeds timeout 120000 ms
24/06/17 01:14:50 WARN SparkContext: Killing executors is not supported by current scheduler.
24/06/17 01:14:56 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$